<a href="https://colab.research.google.com/github/PedroTechy/CarrisInsight/blob/streaming_development/spark_jobs/extract_carris_vehicles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
import utils as utils
from pyspark.sql.functions import  col

In [2]:
spark = SparkSession.builder.appName("StreamingApp").getOrCreate()

25/01/21 00:47:35 WARN Utils: Your hostname, Iness-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.9 instead (on interface en0)
25/01/21 00:47:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/21 00:47:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Step 1: Read and verify results

In [3]:
stops = utils.get_stops(spark)

In [4]:
df = spark.read.format("parquet").load("content/lake/processing/vehicles/data")


In [5]:
df.count()

6979

In [8]:
view = (df
 .orderBy("last_timestamp", ascending=False)
 .dropDuplicates(["id"])
 .join(stops, how='left', on='stop_id')
 .withColumn("distance_till_stop", utils.haversine_udf(col("last_lat"), col('last_lon'), col('stop_lat'), col('stop_lon')))
 .withColumn("stop_eta", col("distance_till_stop") / col("average_speed") *60)
 )


In [9]:
view.show()

25/01/21 00:48:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------+--------------------+--------------------+--------------+--------+-------------------+-------------------+---------+---------+---------+---------+-----------+----------+------------------+---------+---------+------------------+--------------------+
|stop_id|     id|             trip_id|              window|current_status|route_id|    first_timestamp|     last_timestamp|first_lat|first_lon| last_lat| last_lon|   distance|time_delta|     average_speed| stop_lat| stop_lon|distance_till_stop|            stop_eta|
+-------+-------+--------------------+--------------------+--------------+--------+-------------------+-------------------+---------+---------+---------+---------+-----------+----------+------------------+---------+---------+------------------+--------------------+
| 170723|41|1180|1220_0_3_2230_225...|{2025-01-18 22:59...|    STOPPED_AT|  1220_0|2025-01-18 22:59:55|2025-01-18 23:00:16|  38.7494| -9.29466|38.749233|-9.294994|  0.0345108|        21| 5.9161369289670